# Identifying Gypsic Soil in the Guadalupe Mountains
## USGS Landsat 8 Level 2, Collection 2, Tier 1 
### EDS 220, Fall 2021

## Authors: J.A.W.S.

- **J**ulia Parish, UC Santa Barbara (jparish@bren.ucsb.edu)
- **A**lex Vand, UC Santa Barbara (asy@bren.ucsb.edu)
- **W**ylie Hampson, UC Santa Barbara (whampson@bren.ucsb.edu)
- **S**hale Hunter, UC Santa Barbara (shale@bren.ucsb.edu)

## Table of Contents

[1. Purpose](#purpose)

[2. Dataset Description](#overview)

[3. Data I/O](#io)

[4. Metadata Display and Basic Visualization](#display)

[5. Use Case Examples](#usecases)

[6. Create Binder Environment](#binder)

[7. References](#references)

<a id='purpose'></a> 
### Notebook Purpose

This notebook was created to provide an introduction to using satellite data from the <a href="https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C02_T1_L2#description/" target="_blank">USGS Landsat 8 Level 2, Collection 2, Tier 1</a> sourced from Google Earth Engine for analysis to identify soil composition.


<a id='overview'></a> 
### Dataset Description

The dataset JAWS is exploring is <a href="https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C02_T1_L2#description/" target="_blank">USGS Landsat 8 Level 2, Collection 2, Tier 1</a> sourced from Google Earth Engine. Landsat 8 is a joint effort between USGS and NASA, with data collected from 2013-04-11 to the present. The remote sensing equipment on Landsat 8 is the Operational Land Imager (OLI), which is a multichannel scanning radiometer, and a two-channel infared radiometer, Thermal Infrared Sensor (TIRS).  The satellite orbits the earth in 99 minutes, covering individual overlapping swaths, or “scenes”, of the surface. Images are collected over 8-16 days and then compiled to create comprehensive images for the entire earth. Landsat data is represented in WGS84 and the images contain 5 visible and near-infrared bands, 2 short-wave infrared bands, and 1 thermal infrared. This particular dataset is the atmospherically corrected surface reflectance and land surface temperature from the Landsat 8 OLI/TIRS sensors.

Images are provided as TIFs, which can support the wide variety of image types already used for geographic imagery. Metadata files are available for download in Material Library File (MTL) and Extensible Markup Language (XML) formats.


The data retrieval service is Google Earth Engine <a href="https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C02_T1_L2#description/" target="_blank">USGS Landsat 8 Level 2, Collection 2, Tier 1</a>.

There are a few known issues with the Landsat data quality that might be expected to impact the results, but overall it is considered highly accurate and reliable data. One of the most concerning issues is with cloud cover and cloud shadows. The data from this dataset can be affected if there is heavy cloud coverage, due to high albedo/ reflectace of clouds. This may create a bias in the satellite images; however, we show how to account for this later in the notebook. Additionally, the images from the dataset have a 30 meter resolution which will allow us to get high definition images for the area of land that we will be looking at. If you were to look at a small area of land, say a few square kilometers, this data set may not be ideal because the 30 meter resolution won’t allow you to see much detail, but for this use case it works well.

<a id='io'></a> 
### Dataset Input/Output 
Setup for our analysis is quite simple using the Landsat API:

#### 1) Import necessary packages

In [37]:
import ee
import geemap
import pandas as pd
import pprint

In [2]:
ee.Authenticate()
ee.Initialize()

Enter verification code:  4/1AX4XfWj5iQWoePgLCWGechUTwH6j9T5s3Vhw2bntfjUoxBUXUssWfkCjkpY



Successfully saved authorization token.


#### 2) Read in the data from the Landsat API
Note the last part of the identifier indicates that we are using Tier 1 data from NASA's Collection 2 of Landsat data.

In [40]:
# read in Landsat 8 data from Google Earth Engine
data = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")

<a id='display'></a> 
### Metadata Display

To get an idea of what the data comprises, the `propertyNames()` and `getInfo()` functions can be used to display  Landsat 8 metadata. For this project's purposes, we are interested in Landsat 8 bands 7, 6, and 2. A reminder of what wavelengths are recorded in which band can be found [here](https://www.usgs.gov/media/images/landsat-8-band-designations).

An important thing to keep track of with Landsat data is the dates. As satellites cycle through (i.e. break and are replaced), Landsat data must be accessed through different APIs. Landsat 8 data is only availble from February 2013, so if you were interested in images from before then, you would have to access Landsat 7 data, which runs back to April 1999. Landsat data dates back to 1972, when Landsat 1 was launched on July 23, 1972.

#### 1) Display metadata information

Call the `ee.Feature.propertyNames()` function to return the names of the metadata properties of a single image.

In [41]:
# Explore the metadata - metadata display: variables
testImg = data.first()
properties = testImg.propertyNames()
print('Metadata properties:',
      properties.getInfo())  # ee.List of metadata properties

Metadata properties: ['DATA_SOURCE_ELEVATION', 'WRS_TYPE', 'system:id', 'REFLECTANCE_ADD_BAND_1', 'REFLECTANCE_ADD_BAND_2', 'DATUM', 'REFLECTANCE_ADD_BAND_3', 'REFLECTANCE_ADD_BAND_4', 'REFLECTANCE_ADD_BAND_5', 'REFLECTANCE_ADD_BAND_6', 'REFLECTANCE_ADD_BAND_7', 'system:footprint', 'REFLECTIVE_SAMPLES', 'system:version', 'GROUND_CONTROL_POINTS_VERSION', 'SUN_AZIMUTH', 'DATA_SOURCE_TIRS_STRAY_LIGHT_CORRECTION', 'UTM_ZONE', 'DATE_ACQUIRED', 'ELLIPSOID', 'system:time_end', 'DATA_SOURCE_PRESSURE', 'LANDSAT_PRODUCT_ID', 'STATION_ID', 'TEMPERATURE_ADD_BAND_ST_B10', 'DATA_SOURCE_REANALYSIS', 'REFLECTANCE_MULT_BAND_7', 'system:time_start', 'REFLECTANCE_MULT_BAND_6', 'L1_PROCESSING_LEVEL', 'PROCESSING_SOFTWARE_VERSION', 'L1_DATE_PRODUCT_GENERATED', 'ORIENTATION', 'REFLECTANCE_MULT_BAND_1', 'WRS_ROW', 'REFLECTANCE_MULT_BAND_3', 'REFLECTANCE_MULT_BAND_2', 'TARGET_WRS_ROW', 'REFLECTANCE_MULT_BAND_5', 'REFLECTANCE_MULT_BAND_4', 'THERMAL_LINES', 'TIRS_SSM_POSITION_STATUS', 'GRID_CELL_SIZE_THERMAL', 

Utilize the `bandNames()` function to return a list of the bands and their properties. Call the `ee.Feature.getinfo()` imperative function to return metadata information about a feature within the data via an AJAX call. An AJAX call is one method to load personalized content separately from the rest of the HTML document.

In [43]:
# Explore the metadata - metadata display: coordinate information, parameters, information on missing data
pprint.pprint(testImg.getInfo())

{'bands': [{'crs': 'EPSG:32628',
            'crs_transform': [30, 0, 341085, 0, -30, 8808015],
            'data_type': {'max': 65535,
                          'min': 0,
                          'precision': 'int',
                          'type': 'PixelType'},
            'dimensions': [9161, 9161],
            'id': 'SR_B1'},
           {'crs': 'EPSG:32628',
            'crs_transform': [30, 0, 341085, 0, -30, 8808015],
            'data_type': {'max': 65535,
                          'min': 0,
                          'precision': 'int',
                          'type': 'PixelType'},
            'dimensions': [9161, 9161],
            'id': 'SR_B2'},
           {'crs': 'EPSG:32628',
            'crs_transform': [30, 0, 341085, 0, -30, 8808015],
            'data_type': {'max': 65535,
                          'min': 0,
                          'precision': 'int',
                          'type': 'PixelType'},
            'dimensions': [9161, 9161],
            'id': 'SR_B3'}

#### 2) Metadata property definitions

* **'CLOUD_COVER'**: Percentage cloud cover (0-100), -1 = not calculated.
* **'EARTH_SUN_DISTANCE'**: Earth-Sun distance (AU) One astronomical unit (AU) is about 93 million miles (150 million kilometers)
* **'IMAGE_QUALITY_OLI'**: Image quality of the OLI bands. 1 = worst, 9 = best, 0 = quality not calculated. For Landsat 8, this parameter is adjusted downward for scenes collected using the lower 12 bits from the OLI sensor (TRUNCATION_OLI = "LOWER").
* **'IMAGE_QUALITY_TIRS'**: Image quality of the TIRS bands. 1 = worst, 9 = best, 0 = quality not calculated. It is also adjusted downward for scenes processed with "SWITCHED" for the TIRS_SSM_POSITION_STATUS value.
* **'WRS_PATH'**: The Worldwide Reference System (WRS) is a global notation system for Landsat data. It is used to identify the path and row of each Landsat image. The path is the descending orbit of the satellite. Each path is segmented into 119 rows, from north to south. 

### Simple real-color Landsat visualization

#### Set parameters that will be needed for analysis in subsequent portions of the notebook

Here, we create a buffer with a 50km radius around the coordinates (32.05, -104.74) and retrieve the Landsat images that correspond with this area. This region of interest was defined through research by [Stafford et al. (2008)](https://scholarworks.sfasu.edu/cgi/viewcontent.cgi?article=1013&context=geology).

In [36]:
# create a centroid over region of interest: Guadalupe Mountains of New Mexico and Texas
center_lat = 32.05
center_lon = -104.74



To take a look at what the data itself looks like, first select a subset of the data that will have the least interference from cloud cover.

In [6]:
# filter for images with cloud cover < 10%


This subset can be further reduced by selecting a particular timeframe and taking the average of all images in the remaining image collection to leave us with one idealized image to perform our analysis on.

Summer through all months of 2021 were selected in order to examine a particularly sunny and clear time of year. However, there are varied opinions on the best time of year to use. For example, [Browning & Duniway (2011)](https://doi.org/10.1155/2011/421904) chose to use data from February in a similar study in the same part of southern New Mexico because of reduced vegatation cover during winter months (the importance of this will become clear later!).

In [7]:
# Average the cloud filter into a single image within a specified date range and buffer region


In [8]:
# set band parameters using the true color spectrum (4, 3, 2)
visParams = {'bands': [],
             'min': 1,
             'max': 30000
           }

In [9]:
# Basic true color map
map = geemap.Map(center = [center_lat, center_lon], zoom = 7)
map.addLayer(data_mean, visParams)
map

Map(center=[32.05, -104.74], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(child…

<a id='usecase'></a> 
### Use Case Example: Identifying Gypsic Soils

#### Mapping habitat suitability for the identification and conservation of the Guadalupe Mountain necklacepod (*Sophora gypsophila*)

You don't have to be a rocket scientist to use satellite data! This part of the tutorial will focus on an application useful to many conservation ecologists: identifying potential habitat for a threatened species.

_Sophora gypsophila_ is a G1 Critically Endangered shrub in the pea family endemic to a small area surrounding the Guadalupe Mountains in Southeastern New Mexico and Western Texas. Part of what puts _S. gypsophila_ at risk is the fact that it is a substrate obligate - in other words, it can only survive in a specific soil type. But this also gives us an advantage: it means that we can identify potentially suitable habitat simply by mapping the gypsic areas in the vicinity of existing populations of _S. gypsophila_. This can be done using Landsat 8 bands 2, 6, and 7 (Landsat 7 bands 1, 5, and 7), taking advantage of the fact that gypsic soils are highly reflective in band 6 but particularly nonreflective in band 7.


First, we can use the difference between bands 6 and 7 divided by their sum (normalized ratio similar to NDVI calculation) to identify the OIF (Optimum Index Factor) for gypsic soils:

In [10]:
# The bands that we are interested in. 
# SR_B2 (band 2): blue
# SR_B6 (band 6): shortwave infrared 1
# SR_B7 (band 7): shortwave infrared 2


# Optimum index factor statistical value to select the optimum combination of three bands for a satellite image


# Add oif band to our image data


To take a look at our new band, we plot it:

In [11]:
visParams_oif = {'bands': [],
                'min': ,
                'max': ,
                'palette': []
            }

We expect to produce a map that displays white for potential areas with high gypsic content.

In [12]:
map_oif = geemap.Map(center = [center_lat, center_lon], zoom = 7)
map_oif.addLayer(data_oif, visParams_oif)
map_oif

Map(center=[32.05, -104.74], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(child…

As you can see, there is still a problem with this result: some areas that can clearly be identified as `crop circles` show up white in addition to soils with high gypsum content. This means that vegetation is also being detected by our OIF band - not what we want.

To fix this, we can add another band, this time calculating NDVI (Normalized Difference Vegetation Index) with the same math we used to calculate OIF, but using bands 4 (red) and 5 (near-infrared).

In [13]:
red = data_mean.select('SR_B4')
nir = data_mean.select('SR_B5')

ndvi = (nir.subtract(red)).divide((nir.add(red))).rename('NDVI') # Normalized Difference Vegetation Index

data_oif_ndvi = data_oif.addBands(ndvi)

Now, by mapping OIF and NDVI together, we should be able to differentiate between soil and vegetation: this is because vegetation will appear in both bands (show in blue below), but gypsic soils will only appear in OIF (shown in red below).

In [46]:
visParams_oif_ndvi = {'bands': ['OIF', 'NDVI'],
                min: 0,
                max: 0.2
            }

In [47]:
# Plot the OIF band with the NDVI band.




Map(center=[32.05, -104.74], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(child…

If we examine the region with the `crop circles`, we see that this dark vegetated area appears as blue/purple, whereas the nearby areas appear as red. Now, the vegetation and soils are not detected the same way and we are able to distinguish red areas as potential regions of high gypsic content.

#### Results

In summary, this project used the Google Earth Engine API to download Landsat 8 data in order to explore potential habitat for an endangered plant species. By comparing NDVI and OIF bands, this study was able to determine possible regions of high gypsum concentration in the Guadalupe Mountains.

Gypsic soil areas were mapped using the difference ratio of Bands 6 and 7. After plotting the OIF with the NDVI we can see areas that are presumed to be gypics soils standout much more compared to areas that are known to have vegetation.

### Future Research and Other Use Cases

Field surveys could be conducted to determine predictive accuracy. This analysis allows biologist to prioritize areas of potential habitat suitability for survey for rare, threatened species such as _S. gypsophila_.

Furthermore, to calculate more precise estimates for _S. gypsophila_ habitat, we could overlay an elevation map to select only those areas which contain both gypsic soils and fall between 1000-5000 feet above sea level (the known elevation range of current _S. gypsophila_ populations).

An additional step in adapting Landsat data for conservation purposes would be comparing the identified potential _Sophora_ habitat with land ownership maps in the region to target separate management solutions for public and private land.

Federal agencies, renewable energy developers, and biologist may use this analysis to predict areas where rare or threatened plant species may be located but do not have accurate population distribution documentation. 

Other use cases for this analysis or data could be identifying locations for new agricultural areas or for other development. The anaylsis used here would be able to identify areas with high levels of endemism, which could inform environmental impact analysis. 

<a id='references'></a> 
### References

Binte Mostafiz, R.; Noguchi, R.; Ahamed, T. Agricultural Land Suitability Assessment Using Satellite Remote Sensing-Derived Soil-Vegetation Indices. Land 2021, 10, 223. https://doi.org/10.3390/land10020223

Boettinger, J., Ramsey, R., Bodily, J., Cole, N., Kienast-brown, S., Nield, S., Saunders, A., & Stum, A. (2008). Landsat Spectral Data for Digital Soil Mapping. Digital Soil Mapping with Limited Data. https://doi.org/10.1007/978-1-4020-8592-5_16

Browning, D. M., & Duniway, M. C. (2011). Digital Soil Mapping in the Absence of Field Training Data: A Case Study Using Terrain Attributes and Semiautomated Soil Signature Derivation to Distinguish Ecological Potential. Applied and Environmental Soil Science, 2011, e421904. https://doi.org/10.1155/2011/421904

Landsat 8 data courtesy of the U.S. Geological Survey. https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C02_T1_L2#image-properties

Nield, S. J., Boettinger, J. L., & Ramsey, R. D. (2007). Digitally Mapping Gypsic and Natric Soil Areas Using Landsat ETM Data. Soil Science Society of America Journal, 71(1), 245–252. https://doi.org/10.2136/sssaj2006-0049

Ridwan, M. A., Radzi, N., Ahmad, W., Mustafa, I., Din, N., Jalil, Y., Isa, A., Othman A., Zaki, W. Applications of Landsat-8 Data: Survey. Internation Journal of Engineering & Techonology 2018, 11, 436-441. DOI:10.14419/ijet.v7i4.35.22858

Sophora gypsophila | NatureServe Explorer. (n.d.). Retrieved November 16, 2021, from https://explorer.natureserve.org/Taxon/ELEMENT_GLOBAL.2.155365/Sophora_gypsophila

Soil Survey Manual - Ch. 5. Digital Soil Mapping | NRCS Soils. (n.d.). Retrieved November 16, 2021, from https://www.nrcs.usda.gov/wps/portal/nrcs/detail/soils/ref/?cid=nrcs142p2_054255

Stafford, K., Rosales Lagarde, L., & Boston, P. (2008). Castile evaporite karst potential map of the Gypsum Plain, Eddy County New Mexico and Culberson County, Texas: A GIS methodological comparison. J Cave Karst Stud, 70.

Texas Native Plants Database. (n.d.). Retrieved November 16, 2021, from https://aggie-horticulture.tamu.edu/ornamentals/nativeshrubs/sophoragypsophila.htm

Vermote, E., Justice, C., Claverie, M., & Franch, B. (2016). Preliminary analysis of the performance of the Landsat 8/OLI land surface reflectance product. Remote Sensing of Environment, 185, 46-56.